In [1]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
import dask.dataframe as dk
import calendar
from sklearn.metrics import mean_squared_error
from tqdm import tqdm
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler,Normalizer
from scipy.sparse import csr_matrix,hstack
from sklearn.feature_extraction.text import CountVectorizer
import tensorflow as tf
from sklearn.linear_model import SGDRegressor
from sklearn.preprocessing import LabelEncoder
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")
from sklearn.ensemble import AdaBoostRegressor
import pickle

## Data Preprocessing

In [5]:
calendar_df = pd.read_csv("calendar.csv")
train_validation = pd.read_csv("sales_train_validation.csv")
sales_df = pd.read_csv("sell_prices.csv")
train_evaluation = pd.read_csv("sales_train_evaluation.csv")

In [6]:
d_columns = [f'd_{i}' for i in range(1, 1914)]

df_final = pd.melt(train_validation,
                   id_vars=['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'],
                   value_vars=d_columns,
                   var_name="d",
                   value_name="sales")

d_columns = [f'd_{i}' for i in range(1914, 1942)]

df_final_test = pd.melt(train_evaluation,
                        id_vars=['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'],
                        value_vars=d_columns,
                        var_name="d",
                        value_name="sales")

for i in range(1942, 1970):
    train_evaluation[f'd_{i}'] = 0

d_columns = [f'd_{i}' for i in range(1942, 1970)]

df_future_data = pd.melt(train_evaluation,
                         id_vars=['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'],
                         value_vars=d_columns,
                         var_name="d",
                         value_name="sales")

In [7]:
data=df_final.merge(calendar_df,on='d',copy=False)
data=data.merge(sales_df,on=["store_id", "item_id", "wm_yr_wk"],copy=False)
data.to_csv('final_dataframe.csv',index=False)

data_test=df_final_test.merge(calendar_df,on='d',copy=False)
data_test=data_test.merge(sales_df,on=["store_id", "item_id", "wm_yr_wk"],copy=False)
data_test.to_csv('final_dataframe_test.csv',index=False)


data_future=df_future_data.merge(calendar_df,on='d',copy=False)
data_future=data_future.merge(sales_df,on=["store_id", "item_id", "wm_yr_wk"],copy=False)
data_future.to_csv('final_future_data.csv',index=False)

In [16]:
train = data.copy()
test = data_test.copy()
final_test = data_future.copy()

In [11]:
columns_to_encode = [
    'event_name_1', 'event_name_2',
    'event_type_1', 'event_type_2',
    'item_id', 'dept_id',
    'cat_id', 'store_id',
    'state_id', 'year'
]

# Datasets to be transformed
datasets = {
    'train': train,
    'test': test,
    'final_test': final_test
}

# Loop through each column and apply LabelEncoder, then save the encoder
for column in columns_to_encode:
    lbl = LabelEncoder()

    # Fit on train, then transform on all datasets
    datasets['train'][column] = lbl.fit_transform(datasets['train'][column])
    for dataset_name in ['test', 'final_test']:
        datasets[dataset_name][column] = lbl.transform(datasets[dataset_name][column])

    # Save the trained LabelEncoder for each column
    pickle.dump(lbl, open(f'label_encoder_{column}.sav', 'wb'))

In [13]:
datasets = [train, test, final_test]

snap_columns = {
    'CA': 'snap_CA',
    'TX': 'snap_TX',
    'WI': 'snap_WI'
}

# Loop over each dataset and apply the transformations
for dataset in datasets:
    for state_id, snap_col in snap_columns.items():
        # Create the 'snap' column based on the state_id
        dataset.loc[dataset['state_id'] == state_id, 'snap'] = dataset.loc[dataset['state_id'] == state_id][snap_col]

    # Drop the original snap columns
    dataset.drop(['snap_CA', 'snap_TX', 'snap_WI'], axis=1, inplace=True)

In [19]:
dataframes = [train, test, final_test]

columns_to_drop = ['weekday', 'wm_yr_wk']

for df in dataframes:
    df.drop(columns=columns_to_drop, axis=1, inplace=True)

## Feature Engineering

In [20]:
def get_week_number(x):
    date=calendar.datetime.date.fromisoformat(x)
    return date.isocalendar()[1]

train['week_number']=train['date'].apply(lambda x:get_week_number(x))
test['week_number']=test['date'].apply(lambda x:get_week_number(x))
final_test['week_number']=final_test['date'].apply(lambda x:get_week_number(x))

In [21]:
def get_season(x):
    if x in [12,1,2]:
        return 0      #"Winter"
    elif x in [3,4,5]:
        return 1   #"Spring"
    elif x in [6,7,8]:
        return 2   #"Summer"
    else:
        return 3   #"Autumn"


train['season']=train['month'].apply(lambda x:get_season(x))
test['season']=test['month'].apply(lambda x:get_season(x))
final_test['season']=final_test['month'].apply(lambda x:get_season(x))

In [22]:
def check_if_quater_begin(x):
    day=calendar.datetime.date.fromisoformat(x).day
    month=calendar.datetime.date.fromisoformat(x).month
    return 1 if (day==1 and (month in [1,4,7,9])) else 0


train['quater_start']=train['date'].apply(lambda x:check_if_quater_begin(x))
test['quater_start']=test['date'].apply(lambda x:check_if_quater_begin(x))
final_test['quater_start']=final_test['date'].apply(lambda x:check_if_quater_begin(x))

In [23]:
def check_if_quater_end(x):
    day=calendar.datetime.date.fromisoformat(x).day
    month=calendar.datetime.date.fromisoformat(x).month
    if (day==31 and month==3) or (day==30 and month==6) or (day==30 and month==9) or (day==31 and month==12):
        return 1
    else:
        return 0


train['quater_end']=train['date'].apply(lambda x:check_if_quater_end(x))
test['quater_end']=test['date'].apply(lambda x:check_if_quater_end(x))
final_test['quater_end']=final_test['date'].apply(lambda x:check_if_quater_end(x))

In [24]:
def month_start(x):
    day=calendar.datetime.date.fromisoformat(x).day
    return 1 if day==1 else 0


train['month_start']=train['date'].apply(lambda x:month_start(x))
test['month_start']=test['date'].apply(lambda x:month_start(x))
final_test['month_start']=final_test['date'].apply(lambda x:month_start(x))

In [25]:
def month_end(x):
    day=calendar.datetime.date.fromisoformat(x).day
    month=calendar.datetime.date.fromisoformat(x).month
    year=calendar.datetime.date.fromisoformat(x).year
    leap_yr=(year%4==0)
    val=(day==31 and month==1) or (day==29 if leap_yr else day==28) or (day==31 and month==3) or (day==30 and month==4) or\
        (day==31 and month==5) or (day==30 and month==6) or (day==31 and month==7) or (day==31 and month==8) or\
        (day==30 and month==9) or (day==31 and month==10) or (day==30 and month==11) or (day==31 and month==12)
    return 1 if val else 0

train['month_end']=train['date'].apply(lambda x:month_end(x))
test['month_end']=test['date'].apply(lambda x:month_end(x))
final_test['month_end']=final_test['date'].apply(lambda x:month_end(x))

In [26]:
def year_start(x):
    day=calendar.datetime.date.fromisoformat(x).day
    month=calendar.datetime.date.fromisoformat(x).month
    return 1 if (day==1 and month==1) else 0

train['year_start']=train['date'].apply(lambda x:year_start(x))
test['year_start']=test['date'].apply(lambda x:year_start(x))
final_test['year_start']=final_test['date'].apply(lambda x:year_start(x))

In [27]:
def year_end(x):
    day=calendar.datetime.date.fromisoformat(x).day
    month=calendar.datetime.date.fromisoformat(x).month
    return 1 if (day==31 and month==12) else 0

train['year_end']=train['date'].apply(lambda x:year_end(x))
test['year_end']=test['date'].apply(lambda x:year_end(x))
final_test['year_end']=final_test['date'].apply(lambda x:year_end(x))

In [28]:
cv=train[train['date']>='2016-03-28']
train=train[train['date']<'2016-03-28']

In [30]:
import gc

In [31]:
gc.collect()
tt=pd.concat([train,cv,test,final_test])
tt.sort_values(['id','date'],inplace=True)
df=tt.pivot_table(index=['item_id','store_id'],columns='date',values='sales')
df.fillna(0,inplace=True)

In [ ]:
import pandas as pd
import gc

def create_and_merge_rolling_features(df, dataframes, aggregate, shift, r):
    """
    Create rolling feature and merge with given dataframes.

    Args:
    - df: DataFrame on which to calculate rolling features.
    - dataframes: List of DataFrames to merge the rolling feature.
    - aggregate: Aggregation function as a string.
    - shift: Shift periods for rolling window.
    - r: Rolling window size.
    """
    # Calculate rolling features
    roll = df.rolling(window=r, axis=1).agg(aggregate).shift(shift)
    dates = roll.columns
    name = f"roll_{r}_shift_{shift}_{aggregate}"
    roll = roll.astype('float16')
    roll.reset_index(level=[0, 1], inplace=True)
    roll = pd.melt(roll, id_vars=['item_id', 'store_id'], value_vars=dates, var_name='date', value_name=name)
    roll.fillna(-1, inplace=True)

    # Merge with each dataframe in the list
    for dataframe in dataframes:
        dataframe.merge(roll, on=['item_id', 'store_id', 'date'], how='left')
        print(f"Feature created named:= {name}")

    # Cleanup
    del roll
    gc.collect()

# Define parameters for loop
aggregates = ['mean', 'std']
shifts = [28]
rolling_windows = [7, 14, 30, 60, 360]
dataframes_to_merge = [train, cv, final_test, test]  # Assuming these are predefined

# Loop over combinations of aggregates, shifts, and rolling windows
for aggregate in aggregates:
    for shift in shifts:
        for r in rolling_windows:
            create_and_merge_rolling_features(df, dataframes_to_merge, aggregate, shift, r)

Feature created named := roll_7_shift_28_mean
Feature created named := roll_14_shift_28_mean
Feature created named := roll_30_shift_28_mean
Feature created named := roll_60_shift_28_mean
Feature created named := roll_360_shift_28_mean
Feature created named := roll_7_shift_28_std
Feature created named := roll_14_shift_28_std
Feature created named := roll_30_shift_28_std
Feature created named := roll_60_shift_28_std
Feature created named := roll_360_shift_28_std
CPU times: total: 7min 47s
Wall time: 11min 30s


In [33]:
def create_and_merge_ewm_feature(df, dataframes, shift_period, alpha_value, feature_name):
    """
    Create Exponentially Weighted Moving Average (EWMA) feature and merge it with given dataframes.

    Args:
    - df: DataFrame from which to calculate the EWMA feature.
    - dataframes: List of DataFrames to which the EWMA feature will be merged.
    - shift_period: Periods to shift for calculation.
    - alpha_value: Smoothing factor for EWMA.
    - feature_name: Name of the created feature.
    """
    # Shift and calculate EWMA
    ewm_feature = df.shift(shift_period, axis=1).ewm(alpha=alpha_value, axis=1, adjust=False).mean()
    ewm_feature = ewm_feature.astype('float16')
    ewm_feature.reset_index(level=[0, 1], inplace=True)
    ewm_feature = pd.melt(ewm_feature, id_vars=['item_id', 'store_id'], value_vars=ewm_feature.columns, var_name='date', value_name=feature_name)
    ewm_feature.fillna(-1, inplace=True)

    # Merge the feature with each DataFrame in the list
    for dataframe in dataframes:
        dataframe.merge(ewm_feature, on=['item_id', 'store_id', 'date'], how='left')
        print(f"Direct Feature created: {feature_name}")

# Assuming 'df', 'train', 'cv', 'test', and 'final_test' are predefined DataFrames
dataframes_to_merge = [train, cv, test, final_test]

# Call the function with specific parameters
create_and_merge_ewm_feature(df=df, dataframes=dataframes_to_merge, shift_period=28, alpha_value=0.99, feature_name='direct_ewm')


Direct Feature created: direct_ewm
Direct Feature created: direct_ewm
Direct Feature created: direct_ewm
Direct Feature created: direct_ewm


In [ ]:
def create_lag_feature(df, lag):
    """
    Creates a lag feature for a given DataFrame and lag value.

    Parameters:
    - df: The DataFrame to shift.
    - lag: The lag value.

    Returns:
    - A DataFrame containing the lag feature.
    """
    i = 'direct_lag_' + str(lag)
    lag_i = df.shift(lag, axis=1)
    dates = lag_i.columns
    lag_i.reset_index(level=[0, 1], inplace=True)
    lag_i = pd.melt(lag_i, id_vars=['item_id', 'store_id'], value_vars=dates, var_name='date', value_name=i)
    lag_i.fillna(-1, inplace=True)
    lag_i[i] = lag_i[i].astype('int16')
    return lag_i

def merge_lag_feature(datasets, lag_feature):
    """
    Merges the lag feature into the specified datasets.

    Parameters:
    - datasets: A list of DataFrames to merge the lag feature into.
    - lag_feature: The lag feature DataFrame to merge.
    """
    for i in range(len(datasets)):
        datasets[i] = datasets[i].merge(lag_feature, on=['item_id', 'store_id', 'date'])

# Assuming 'df' is your initial DataFrame and train, cv, test, final_test are defined
for lag in range(28, 100, 7):
    lag_feature = create_lag_feature(df, lag)
    merge_lag_feature([train, cv, test, final_test], lag_feature)
    print("Feature created for lag", lag)
    del lag_feature
    gc.collect()

Feature created for lag 28
Feature created for lag 35
Feature created for lag 42
Feature created for lag 49
Feature created for lag 56
Feature created for lag 63
Feature created for lag 70
Feature created for lag 77
Feature created for lag 84
Feature created for lag 91
Feature created for lag 98
CPU times: total: 8min 21s
Wall time: 12min 58s


In [ ]:
# train.to_csv('train_df.csv',index=False)
# cv.to_csv('cross_validation_df.csv',index=False)
# test.to_csv('test_df.csv',index=False)
# final_test.to_csv('final_test_df.csv',index=False)

CPU times: total: 2min 52s
Wall time: 6min 26s
